# 1 Imports

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import glob

# 2 Connect to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPEN_API_KEY')
model = "gpt-4o-mini"

In [3]:
openai = OpenAI()

# 3 Import Knowledge Base

In [4]:
context = {}

employees = glob.glob("knowledge-base/employees/*")

for employee in employees:

    name = employee.split(' ')[-1][:-3]

    with open(employee, 'r', encoding="utf-8") as f:
        doc = f.read()

    context[name] = doc

In [5]:
products = glob.glob("knowledge-base/products/*")

for product in products:
    
    name = product.split('\\')[-1][:-3]

    with open(product, 'r', encoding="utf-8") as f:
        doc = f.read()

    context[name] = doc

# 4 OpenAI Model Configuration

In [6]:
system_message = '''You are an expert in answering accurate questions about employees about Insurellm, the Isurance Tech company. Give brief, accurate answer.
 If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context.
'''

In [7]:
def get_relevant_context(message):
    relevant_context = []

    for context_title, context_details in context.items():
        if context_title in message:
            relevant_context.append(context_details)
    return relevant_context

In [8]:
def add_context(message):
    relevant_context = get_relevant_context(message)

    if relevant_context:
        message += "\n\n The following additional context might be relevant in answering this question: \n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
            
    return message

In [9]:
print(add_context("Whos is Avery Lancaster?"))

Whos is Avery Lancaster?

 The following additional context might be relevant in answering this question: 

# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985  
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  
- **Location**: San Francisco, California  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  
  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  

- **2010 - 2013**: Business Analyst at Edge Analytics  
  Prior to joining In

In [10]:
def chat(message, history):
    
    messages = [{'role': 'system', 'content': system_message}]
    for user_message, assistant_message in history:
        messages.append({'role': 'user', 'content': user_message})
        messages.append({'role': 'assistant', 'content': assistant_message})

    message = add_context(message)
    messages.append({'role': 'user', 'content': message})

    stream = openai.chat.completions.create(model=model, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# 5 UI with Gradio

In [11]:
view = gr.ChatInterface(chat).launch()

c:\Users\filip\GitHub\Mastering-LLM-Agentic-AI\venv\Lib\site-packages\gradio\components\chatbot.py:290: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
